# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests

# Import API key
from api_keys import g_key
from api_keys import weather_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the csv into a dataframe
cities_df = pd.read_csv('./output_data/cities.csv', encoding="utf-8")
# drop any na rows
cities_df.dropna()
# Check the dataframe by examining just the head
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mahébourg,40,MU,1584135073,94,-20.41,57.70,75.00,3.36
1,Tuktoyaktuk,75,CA,1584135073,85,69.45,-133.04,17.60,25.28
2,Manta,20,EC,1584135073,66,-0.95,-80.73,84.20,11.41
3,Kwang Binh,90,VN,1584135073,96,17.48,106.60,73.49,4.56
4,Jamestown,1,US,1584135073,48,42.10,-79.24,46.40,27.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
# Store humidity information in humidity
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
coordinates = cities_df[["Lat", "Lng"]]

In [5]:
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout,center=(0.0, 0.0), zoom_level=1.8)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius=1)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(coordinates, weights=humidity))

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 65.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
hotel_df =   cities_df[(cities_df['Max Temp'] >= 76) 
          & (cities_df['Max Temp'] < 80) 
          & (cities_df['Wind Speed'] < 10)
          & (cities_df['Cloudiness'] == 0)].reset_index()

# set all the values to '' for hotel name
hotel_df.insert(loc=1, column='Hotel Name', value=['' for i in range(hotel_df.shape[0])])
#hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each row finding the nearest hotel by lat,lng coordinates
# and populate the 'Hotel Name' column with the closest hotel name
# The hotel names are also printed to the screen

for index, row in hotel_df.iterrows():
    # put the lat,lng values into string location
    # so it can help build the query
    location = f"{row['Lat']}, {row['Lng']}"
    # set up parameters
    params = {
        "location": location,  # city coords,
        "rankby": "distance",
        "type": "lodging",
        "key": g_key,
    }
    
    # print(params)   # for debugging    
    # make the request
    response = requests.get(base_url, params=params).json()
   
    # extract results
    results = response['results']
       
    try:
        print(f"The closest hotel to the city of {row['City']} is the {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = {results[0]['name']}      
    except (KeyError, IndexError):
        print("*** Missing field/result... skipping entry. *** ")
        
    print("------------------------------------------------------------")
    
#hotel_df['Hotel Name']
#hotel_df

*** Missing field/result... skipping entry. *** 
------------------------------------------------------------
The closest hotel to the city of Beira is the SCI Response Hub Office.
------------------------------------------------------------
The closest hotel to the city of Cidreira is the Pousada 591.
------------------------------------------------------------
The closest hotel to the city of Pandan is the Bonifacio Lodge.
------------------------------------------------------------
The closest hotel to the city of Moa is the Renta en Divisa.
------------------------------------------------------------
*** Missing field/result... skipping entry. *** 
------------------------------------------------------------
The closest hotel to the city of Pompéia is the Isabel Sgorlon Mascarin.
------------------------------------------------------------
The closest hotel to the city of Guiglo is the Mission Internationale La Grande Moisson Guiglo.
------------------------------------------------

In [8]:
hotel_df['Hotel Name'].replace('', np.nan,  inplace = True)
hotel_df = hotel_df.dropna( ).reset_index()
# hotel_df

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#locations  # for debugging
    

In [10]:
len(hotel_df['Hotel Name'])

9

In [11]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hotel_df['Hotel Name'])
# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### End of notebook